# Animation Slider cut edition
---

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import mpl_toolkits.axes_grid1
import matplotlib.widgets

In [ ]:
%matplotlib qt

In [ ]:
class Player(FuncAnimation):
    def __init__(self, fig, func, frames=None, init_func=None,fps=30, fargs=None,
                 save_count=None, mini=0, maxi=100, pos=(0.125, 0.92), **kwargs):
        self.i = 0
        self.min=mini
        self.max=maxi
        self.runs = True
        self.forwards = True
        self.fig = fig
        self.func = func
        self.setup(pos)
        FuncAnimation.__init__(self,self.fig, self.update, frames=self.play(), 
                                           init_func=init_func, fargs=fargs,interval=1000/fps,
                                           save_count=save_count, **kwargs )    

    def play(self):
        while self.runs:
            self.i = self.i+self.forwards-(not self.forwards)
            if self.i > self.min and self.i < self.max:
                yield self.i
            else:
                self.stop()
                yield self.i

    def start(self):
        self.runs=True
        self.event_source.start()

    def stop(self, event=None):
        self.runs = False
        self.event_source.stop()

    def forward(self, event=None):
        self.forwards = True
        self.start()
    def backward(self, event=None):
        self.forwards = False
        self.start()
    def oneforward(self, event=None):
        self.forwards = True
        self.onestep()
    def onebackward(self, event=None):
        self.forwards = False
        self.onestep()

    def onestep(self):
        if self.i > self.min and self.i < self.max:
            self.i = self.i+self.forwards-(not self.forwards)
        elif self.i == self.min and self.forwards:
            self.i+=1
        elif self.i == self.max and not self.forwards:
            self.i-=1
        self.func(self.i)
        self.slider.set_val(self.i)
        self.fig.canvas.draw_idle()

    def setup(self, pos):
        playerax = self.fig.add_axes([pos[0],pos[1], 0.64, 0.04])
        divider = mpl_toolkits.axes_grid1.make_axes_locatable(playerax)
        bax = divider.append_axes("right", size="80%", pad=0.05)
        sax = divider.append_axes("right", size="80%", pad=0.05)
        fax = divider.append_axes("right", size="80%", pad=0.05)
        ofax = divider.append_axes("right", size="100%", pad=0.05)
        sliderax = divider.append_axes("right", size="500%", pad=0.07)
        self.button_oneback = matplotlib.widgets.Button(playerax, label='$\u29CF$')
        self.button_back = matplotlib.widgets.Button(bax, label='$\u25C0$')
        self.button_stop = matplotlib.widgets.Button(sax, label='$\u25A0$')
        self.button_forward = matplotlib.widgets.Button(fax, label='$\u25B6$')
        self.button_oneforward = matplotlib.widgets.Button(ofax, label='$\u29D0$')
        self.button_oneback.on_clicked(self.onebackward)
        self.button_back.on_clicked(self.backward)
        self.button_stop.on_clicked(self.stop)
        self.button_forward.on_clicked(self.forward)
        self.button_oneforward.on_clicked(self.oneforward)
        self.slider = matplotlib.widgets.Slider(sliderax, '', 
                                                self.min, self.max, valinit=self.i)
        self.slider.on_changed(self.set_pos)

    def set_pos(self,i):
        self.i = int(self.slider.val)
        self.func(self.i)

    def update(self,i):
        self.slider.set_val(i)



In [ ]:
from ClementAnalyse_animation_work import *
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import cmasher as cmr

In [ ]:

def animate_data(
    test,
    xedge_setting,
    yedge_setting,
    step=20000,
    cmap=cmr.cosmic,
    vmax=25,
    vmin=0,
    begin=0,
    fps= 2
    ):
    interval = 1000/fps
    # Define the initial values
    begin = begin
    end = begin + step
    bins = (xedge_setting, yedge_setting)
    all_counts = []

    # Define the figure and axis objects
    fig, ax = plt.subplots()
    cbar = None

    # Define the update function that will be called for each frame of the animation
    def update(frame):
        nonlocal cbar, all_counts, begin, end
        if cbar is not None:
            cbar.remove()

        # Extract the slice of the data for this frame
        begin = frame * step
        end = begin + step
        slice = test[(test.TIME > begin) & (test.TIME < end)]

        # Compute the histogram for this slice
        count, _, _ = np.histogram2d(slice.L_cut, slice.B_cut, bins=bins)

        # Add the counts to the list of all counts
        all_counts.append(count)

        # Plot the current frame
        ax.clear()
        pcm = ax.imshow(all_counts[frame].T, extent=[xedge_setting[0], xedge_setting[-1], yedge_setting[0], yedge_setting[-1]],
                        origin='lower', cmap=cmap, vmax=vmax,vmin=vmin, interpolation='None', aspect='equal')
        ax.set_title(f'Step={step/60} min ({np.round(step/3600,1)}h) \n Time :\n {np.round(slice.TIME.min()/(24*3600),1)}j \n {np.round((slice.TIME.max()/(24*3600)),1)} j')

        # Add a colorbar to the plot
        cbar = fig.colorbar(pcm)

    # Create the animation object and start the animation
    num_frames = int(test.TIME.max()+1)//step
    # ani = FuncAnimation(fig, update, frames=num_frames, interval=interval,repeat =True)
    # plt.show()
    return fig, update,num_frames,interval

## Chemin des données

### Interface pour demander la position du repertoire courant

🚩🚩 Avec la fenaitre renseigner la position du fichier Fits à analysé 🚩🚩
Une fois appuyer sur set path vous pouvez fermer la fenaitre

In [ ]:
from tkinter import *
from tkinter import ttk
from tkinter import filedialog
import os
gui = Tk()
gui.geometry("400x400")
gui.title("Recherche des fichier Fits")

def getFolderPath():
    folder_selected = filedialog.askopenfilename()
    folderPath.set(folder_selected)
Path = ""

def doStuff():
    folder = folderPath.get()
    global Path
    Path = folder
    print("Fits Path set", folder)
    return Path

folderPath = StringVar()
a = Label(gui ,text="Chemin Fichier FITS")
a.grid(row=0,column = 0)
E = Entry(gui,textvariable=folderPath)
E.grid(row=0,column=1)
btnFind = ttk.Button(gui, text="Browse Folder",command=getFolderPath)
btnFind.grid(row=0,column=2)

c = ttk.Button(gui ,text="\t Set Path \n-\n(you can close after with [x] butom)", command=doStuff)
c.grid(row=4,column=0)
gui.mainloop()

week_path = Path

In [ ]:
# week_path = "HauteEnergie/Data/source_observation/lat_photon_weekly_w015_p305_v001.fits" #Chemin du fichier FITS

## Execution de l'animation

In [ ]:
step=3*3600 # Combien de seconde corespon une image ex: 3*3600 soit 3H de donné cumulé
fps = 30 # Image par seconde pour l'animation
seuil_max = 10 #seuil supérieur pour le décompte
seuil_min = 0 # --- inf
cmap = cmr.cosmic 

In [ ]:
# Traitement des données
data = GammaFitsPandas(week_path)
plt.close() #ferme la figure issue de GammaFits
week = data.events #Récupère les donné sous forme de pandas DataFrame
tmax = int(np.round(week.TIME.max())) #Calcule le temps max des données

# Calcule la taille de chaque bin et garde xedges et yedges en mémoir pour rendre le maillage uniforme entre chaque coupe
data.keep_bin_edge 
# Extraction des bord des edges
xedge_setting = data.xedges_tot
yedge_setting = data.yedges_tot
# plt.close('all')

# Création et récupération des élément animable
fig, update,num_frames,interval=animate_data(
    week,
    xedge_setting,
    yedge_setting,
    step=step,
    vmax=seuil_max,
    vmin=seuil_min,
    begin=0,
    fps=fps,
    cmap=cmap
    )

# Animation dans le player
ani = Player(fig,
             update,
             frames=num_frames,
             maxi=tmax//(step),
             fps=fps)
# Affichage

plt.show()